In [1]:
import os
import json
import datetime
import logging
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import ToolMessage
from langchain.tools import Tool
from order_handler import OrderHandler
from db import Database
from IPython.display import Image, display
from config import GROK_API_KEY

In [2]:
# ✅ Load environment variables
load_dotenv()

True

In [3]:
# ✅ Initialize the LLM
llm = ChatGroq(
    temperature=0.3,
    groq_api_key=GROK_API_KEY,
    model_name="qwen-2.5-32b"
)

In [4]:
# ✅ Define a TypedDict to store chat messages
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [85]:
# ✅ Define the functions for each tool
def show_menu(_=None):
    """Fetch and display the menu items."""
    menu = OrderHandler().fetch_menu()
    if not menu:
        return "Sorry, the menu is currently unavailable."
    
    return f"{menu}"

In [86]:
show_menu()

"{'Cheese Burger': 5.99, 'Chicken Burger': 6.99, 'Veggie Burger': 5.49, 'Pepperoni Pizza': 12.99, 'Margherita Pizza': 11.49, 'BBQ Chicken Pizza': 13.99, 'Grilled Chicken Sandwich': 7.99, 'Club Sandwich': 6.99, 'Spaghetti Carbonara': 9.99, 'Fettuccine Alfredo': 10.49, 'Tandoori Chicken': 11.99, 'Butter Chicken': 12.49, 'Beef Steak': 15.99, 'Chicken Biryani': 8.99, 'Mutton Biryani': 10.99, 'Prawn Curry': 13.49, 'Fish and Chips': 9.49, 'French Fries': 3.99, 'Garlic Bread': 4.49, 'Chocolate Brownie': 5.49, 'Vanilla Ice Cream': 3.99, 'Strawberry Shake': 4.99, 'Mango Smoothie': 5.49, 'Coca-Cola': 2.49, 'Pepsi': 2.49, 'Fresh Orange Juice': 4.99}"

In [7]:
# ✅ Define tools with descriptions
tools = [
    Tool(name="Show Menu", func=show_menu, description="Fetch and display the menu items."),
]

In [8]:
# ✅ Define Tool Execution Class
class ToolExecutor:
    """Handles tool execution when called by the LLM."""
    
    def __init__(self, tools: list):
        self.tools_by_name = {tool.name: tool for tool in tools}
    
    def __call__(self, state: State):
        messages = state.get("messages", [])
        last_message = messages[-1] if messages else None
        
        if not last_message or not hasattr(last_message, "tool_calls"):
            return {"messages": messages}
        
        tool_results = []
        for tool_call in last_message.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]
            
            if tool_name in self.tools_by_name:
                tool_result = self.tools_by_name[tool_name].invoke(tool_args)
                tool_results.append(ToolMessage(
                    content=json.dumps(tool_result),
                    tool=tool_name,
                    tool_call_id=tool_call["id"]
                ))
        
        return {"messages": messages + tool_results}

In [9]:
# ✅ Define Routing Function
def route_tools(state: State):
    """Routes the flow based on whether tools are required."""
    messages = state.get("messages", [])
    
    if messages and hasattr(messages[-1], "tool_calls") and messages[-1].tool_calls:
        return "tools"
    
    return END

In [10]:
# ✅ Build the LangGraph Chatbot Flow
graph_builder = StateGraph(State)
llm_with_tools = llm.bind_tools(tools)

In [11]:
def chatbot(state: State):
    """Handles AI response and tool calling."""
    ai_response = llm_with_tools.invoke(state["messages"])
    return {"messages": [ai_response]}

In [12]:
# ✅ Add nodes to the graph
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolExecutor(tools))

# ✅ Define edges (flow)
graph_builder.add_conditional_edges("chatbot", route_tools, {"tools": "tools", END: END})
graph_builder.add_edge("tools", "chatbot")  # Return to chatbot after tool execution
graph_builder.add_edge(START, "chatbot")

# ✅ Compile the graph
graph = graph_builder.compile()

In [13]:
# ✅ Function to Stream Responses
def stream_graph_updates(user_input: str):
    """Streams chatbot responses from LangGraph."""
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

In [38]:
show_menu()

"Here's our menu:\nCheese Burger - $5.99\nChicken Burger - $6.99\nVeggie Burger - $5.49\nPepperoni Pizza - $12.99\nMargherita Pizza - $11.49\nBBQ Chicken Pizza - $13.99\nGrilled Chicken Sandwich - $7.99\nClub Sandwich - $6.99\nSpaghetti Carbonara - $9.99\nFettuccine Alfredo - $10.49\nTandoori Chicken - $11.99\nButter Chicken - $12.49\nBeef Steak - $15.99\nChicken Biryani - $8.99\nMutton Biryani - $10.99\nPrawn Curry - $13.49\nFish and Chips - $9.49\nFrench Fries - $3.99\nGarlic Bread - $4.49\nChocolate Brownie - $5.49\nVanilla Ice Cream - $3.99\nStrawberry Shake - $4.99\nMango Smoothie - $5.49\nCoca-Cola - $2.49\nPepsi - $2.49\nFresh Orange Juice - $4.99"

In [14]:
stream_graph_updates("menu")

Assistant: 
Assistant: "Here's our menu:\nCheese Burger - $5.99\nChicken Burger - $6.99\nVeggie Burger - $5.49\nPepperoni Pizza - $12.99\nMargherita Pizza - $11.49\nBBQ Chicken Pizza - $13.99\nGrilled Chicken Sandwich - $7.99\nClub Sandwich - $6.99\nSpaghetti Carbonara - $9.99\nFettuccine Alfredo - $10.49\nTandoori Chicken - $11.99\nButter Chicken - $12.49\nBeef Steak - $15.99\nChicken Biryani - $8.99\nMutton Biryani - $10.99\nPrawn Curry - $13.49\nFish and Chips - $9.49\nFrench Fries - $3.99\nGarlic Bread - $4.49\nChocolate Brownie - $5.49\nVanilla Ice Cream - $3.99\nStrawberry Shake - $4.99\nMango Smoothie - $5.49\nCoca-Cola - $2.49\nPepsi - $2.49\nFresh Orange Juice - $4.99"
Assistant: Here's our menu:

- Cheese Burger - $5.99
- Chicken Burger - $6.99
- Veggie Burger - $5.49
- Pepperoni Pizza - $12.99
- Margherita Pizza - $11.49
- BBQ Chicken Pizza - $13.99
- Grilled Chicken Sandwich - $7.99
- Club Sandwich - $6.99
- Spaghetti Carbonara - $9.99
- Fettuccine Alfredo - $10.49
- Tandoo

In [15]:
# ✅ Start Chatbot Loop
print("\n🤖 Chatbot is ready! Type 'exit', 'quit', or 'bye' to stop.\n")

while True:
    try:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except Exception as e:
        print("Error:", str(e))


🤖 Chatbot is ready! Type 'exit', 'quit', or 'bye' to stop.

Assistant: Hello! How can I assist you today?
Assistant: 
Assistant: "Here's our menu:\nCheese Burger - $5.99\nChicken Burger - $6.99\nVeggie Burger - $5.49\nPepperoni Pizza - $12.99\nMargherita Pizza - $11.49\nBBQ Chicken Pizza - $13.99\nGrilled Chicken Sandwich - $7.99\nClub Sandwich - $6.99\nSpaghetti Carbonara - $9.99\nFettuccine Alfredo - $10.49\nTandoori Chicken - $11.99\nButter Chicken - $12.49\nBeef Steak - $15.99\nChicken Biryani - $8.99\nMutton Biryani - $10.99\nPrawn Curry - $13.49\nFish and Chips - $9.49\nFrench Fries - $3.99\nGarlic Bread - $4.49\nChocolate Brownie - $5.49\nVanilla Ice Cream - $3.99\nStrawberry Shake - $4.99\nMango Smoothie - $5.49\nCoca-Cola - $2.49\nPepsi - $2.49\nFresh Orange Juice - $4.99"
Assistant: Here's our menu:

- Cheese Burger - $5.99
- Chicken Burger - $6.99
- Veggie Burger - $5.49
- Pepperoni Pizza - $12.99
- Margherita Pizza - $11.49
- BBQ Chicken Pizza - $13.99
- Grilled Chicken Sa

In [112]:
menu = show_menu()

In [114]:
menu

"{'Cheese Burger': 5.99, 'Chicken Burger': 6.99, 'Veggie Burger': 5.49, 'Pepperoni Pizza': 12.99, 'Margherita Pizza': 11.49, 'BBQ Chicken Pizza': 13.99, 'Grilled Chicken Sandwich': 7.99, 'Club Sandwich': 6.99, 'Spaghetti Carbonara': 9.99, 'Fettuccine Alfredo': 10.49, 'Tandoori Chicken': 11.99, 'Butter Chicken': 12.49, 'Beef Steak': 15.99, 'Chicken Biryani': 8.99, 'Mutton Biryani': 10.99, 'Prawn Curry': 13.49, 'Fish and Chips': 9.49, 'French Fries': 3.99, 'Garlic Bread': 4.49, 'Chocolate Brownie': 5.49, 'Vanilla Ice Cream': 3.99, 'Strawberry Shake': 4.99, 'Mango Smoothie': 5.49, 'Coca-Cola': 2.49, 'Pepsi': 2.49, 'Fresh Orange Juice': 4.99}"

In [115]:
order_dict = {"Pepsi": 2, "Coca-cola": 4}
order_items = {}

In [120]:
response = ""
total_price = float(0.0)
response = ""
for item, quantity in order_dict.items():
    item_lower = item.lower()
    if item_lower in menu.lower():
        if item_lower in order_items:
            order_items[item_lower] += quantity  # Update quantity
        else:
            order_items[item_lower] = quantity  # Add new item
        # total_price += menu[item_lower] * quantity
        response += f"✅ Added {quantity}x {item}.\n"
    else:
        response += f"⚠ {item} is unavailable.\n"

In [121]:
order_items

{'pepsi': 4, 'coca-cola': 8}

In [2]:
item = tuple(('pepsi', 'coca-cola'))

In [ ]:
({"old_item":item[0], "new_item":item[1]})

TypeError: the JSON object must be str, bytes or bytearray, not dict